# Assignment 2.1
### Scrape and Analyse

* API [https://beautiful-soup-4.readthedocs.io/en/latest/](https://beautiful-soup-4.readthedocs.io/en/latest/)

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Tasks
Scrape data from the website [http://www.nationmaster.com](http://www.nationmaster.com/), convert it into Pandas data frames and use pandas queries to answer the following questions: 

#### 1
Get the number of internet users per country, remove all NaN entries and return the top 10 countries with the highest absolute number of internet users. 

In [47]:
#use request to get data from URL
res = requests.get("http://www.nationmaster.com/country-info/stats/Media/Internet-users")
#parse data as HTML
soup = BeautifulSoup(res.content,'lxml')
#extract tables
table = soup.find_all('table')[0]
#convert table to pandas data frame
df = pd.read_html(str(table)) #returns list of dataframes (one for each table)

In [48]:
df0 = df[0].drop(columns=['GRAPH', 'HISTORY'])

In [53]:
df0 = df0.dropna(subset=['#','COUNTRY','AMOUNT'])

In [55]:
df0 = df0.drop(columns=['#'])

In [56]:
df0

,COUNTRY,AMOUNT,DATE
0,China,389 million,2009
1,United States,245 million,2009
2,Japan,99.18 million,2009
4,Brazil,75.98 million,2009
5,Germany,65.12 million,2010
...,...,...,...
244,Niue,1100,2009
245,"Saint Helena, Ascension, and Tristan da Cunha",900,2009
246,Saint Helena,900,2009
247,Tokelau,800,2008


In [57]:
df0['AMOUNT'] = df0['AMOUNT'].replace({'million': '*1e6'}, regex=True)

In [58]:
df0['AMOUNT'] = df0['AMOUNT'].map(pd.eval)

In [59]:
df0.sort_values(by=['AMOUNT'], ascending=False)

,COUNTRY,AMOUNT,DATE
0,China,389000000.0,2009
1,United States,245000000.0,2009
2,Japan,99180000.0,2009
4,Brazil,75980000.0,2009
5,Germany,65120000.0,2010
...,...,...,...
244,Niue,1100.0,2009
245,"Saint Helena, Ascension, and Tristan da Cunha",900.0,2009
246,Saint Helena,900.0,2009
247,Tokelau,800.0,2008


In [60]:
df0.head(10)

,COUNTRY,AMOUNT,DATE
0,China,389000000.0,2009
1,United States,245000000.0,2009
2,Japan,99180000.0,2009
4,Brazil,75980000.0,2009
5,Germany,65120000.0,2010
6,India,61340000.0,2009
7,Russia,59700000.0,2010
9,United Kingdom,51440000.0,2009
10,France,44630000.0,2010
11,Nigeria,43990000.0,2009


#### 2
Get the number of internet users per country, remove all NaN entries and return the top 10 countries with the highest number of internet users relative to the populutation. Hint: you need to scrape the population number from another page)

In [76]:
#use request to get data from URL
res = requests.get("http://www.nationmaster.com/country-info/stats/People/Population")
#parse data as HTML
soup = BeautifulSoup(res.content,'lxml')
#extract tables
table = soup.find_all('table')[0]
#convert table to pandas data frame
df_pop_list = pd.read_html(str(table)) #returns list of dataframes (one for each table)

In [77]:
df_pop = df_pop_list[0]

In [78]:
df_pop = df_pop.drop(columns=['GRAPH', 'HISTORY'])
df_pop = df_pop.dropna(subset=['#','COUNTRY','AMOUNT'])
df_pop = df_pop.drop(columns=['#'])

In [79]:
df_pop

,COUNTRY,AMOUNT,DATE
0,China,1.35 billion,2013
1,India,1.22 billion,2013
2,European Union,509.04 million,2012
4,United States,316.67 million,2013
5,Indonesia,251.16 million,2013
...,...,...,...
296,Coral Sea Islands,0.0,2013
297,Clipperton Island,0.0,2013
298,Wake Island,0.0,2013
299,Spratly Islands,0.0,2013


In [85]:
df_pop['AMOUNT'] = df_pop['AMOUNT'].replace({'million': '*1e6','billion': '*1e9'}, regex=True).map(pd.eval)

In [86]:
result = pd.merge(df0, df_pop, on="COUNTRY")

In [87]:
result

,COUNTRY,AMOUNT_x,DATE_x,AMOUNT_y,DATE_y
0,China,389000000.0,2009,1.350000e+09,2013
1,United States,245000000.0,2009,3.166700e+08,2013
2,Japan,99180000.0,2009,1.272500e+08,2013
3,Brazil,75980000.0,2009,2.010100e+08,2013
4,Germany,65120000.0,2010,8.115000e+07,2013
...,...,...,...,...,...
213,Niue,1100.0,2009,1.229000e+03,2013
214,"Saint Helena, Ascension, and Tristan da Cunha",900.0,2009,7.754000e+03,2013
215,Saint Helena,900.0,2009,7.700000e+03,2011
216,Tokelau,800.0,2008,1.353000e+03,2013


In [88]:
result['relative'] = result.apply(lambda x: x['AMOUNT_x'] / x['AMOUNT_y'], axis=1)

In [89]:
result

,COUNTRY,AMOUNT_x,DATE_x,AMOUNT_y,DATE_y,relative
0,China,389000000.0,2009,1.350000e+09,2013,0.288148
1,United States,245000000.0,2009,3.166700e+08,2013,0.773676
2,Japan,99180000.0,2009,1.272500e+08,2013,0.779411
3,Brazil,75980000.0,2009,2.010100e+08,2013,0.377991
4,Germany,65120000.0,2010,8.115000e+07,2013,0.802465
...,...,...,...,...,...,...
213,Niue,1100.0,2009,1.229000e+03,2013,0.895037
214,"Saint Helena, Ascension, and Tristan da Cunha",900.0,2009,7.754000e+03,2013,0.116069
215,Saint Helena,900.0,2009,7.700000e+03,2011,0.116883
216,Tokelau,800.0,2008,1.353000e+03,2013,0.591279


In [91]:
result.sort_values(by=['relative'], ascending=False)

,COUNTRY,AMOUNT_x,DATE_x,AMOUNT_y,DATE_y,relative
128,Iceland,301600.0,2010,315281.0,2013,0.956607
54,Norway,4430000.0,2010,4720000.0,2013,0.938559
208,Falkland Islands (Islas Malvinas),2900.0,2009,3140.0,2008,0.923567
33,Sweden,8400000.0,2010,9120000.0,2013,0.921053
213,Niue,1100.0,2009,1229.0,2013,0.895037
...,...,...,...,...,...,...
131,Democratic Republic of the Congo,290000.0,2008,75510000.0,2013,0.003841
111,Bangladesh,617300.0,2009,163650000.0,2013,0.003772
198,Sierra Leone,14900.0,2009,5610000.0,2013,0.002656
157,Burma,110000.0,2009,55170000.0,2013,0.001994


In [92]:
result.head(10)

,COUNTRY,AMOUNT_x,DATE_x,AMOUNT_y,DATE_y,relative
0,China,389000000.0,2009,1.350000e+09,2013,0.288148
1,United States,245000000.0,2009,3.166700e+08,2013,0.773676
2,Japan,99180000.0,2009,1.272500e+08,2013,0.779411
3,Brazil,75980000.0,2009,2.010100e+08,2013,0.377991
4,Germany,65120000.0,2010,8.115000e+07,2013,0.802465
5,India,61340000.0,2009,1.220000e+09,2013,0.050279
6,Russia,59700000.0,2010,1.425000e+08,2013,0.418947
7,United Kingdom,51440000.0,2009,6.318000e+07,2014,0.814182
8,France,44630000.0,2010,6.595000e+07,2013,0.676725
9,Nigeria,43990000.0,2009,1.745100e+08,2013,0.252077


#### 3
Compute the correlation between the crime rate (murders per 100k) and the education level. Compare this to the correlation of crime rate and poverty (relative BIP). Hint: use pandas build in correlation function: [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)

In [125]:
#use request to get data from URL
res = requests.get("https://www.nationmaster.com/country-info/stats/Crime/Violent-crime/Murder-rate")
#parse data as HTML
soup = BeautifulSoup(res.content,'lxml')
#extract tables
table = soup.find_all('table')[0]
#convert table to pandas data frame
df_murder_list = pd.read_html(str(table)) #returns list of dataframes (one for each table)

In [126]:
df_murder = df_murder_list[0]

In [127]:
#use request to get data from URL
res = requests.get("https://www.nationmaster.com/country-info/stats/Education/High-school-enrolment-rate")
#parse data as HTML
soup = BeautifulSoup(res.content,'lxml')
#extract tables
table = soup.find_all('table')[0]
#convert table to pandas data frame
df_edu_list = pd.read_html(str(table)) #returns list of dataframes (one for each table)

In [128]:
df_edu = df_edu_list[0]

In [129]:
df_murder = df_murder.drop(columns=['GRAPH', 'HISTORY','DATE'])
df_murder = df_murder.dropna(subset=['#','COUNTRY','AMOUNT'])
df_murder = df_murder.drop(columns=['#'])

df_edu = df_edu.drop(columns=['GRAPH', 'HISTORY','DATE'])
df_edu = df_edu.dropna(subset=['#','COUNTRY','AMOUNT'])
df_edu = df_edu.drop(columns=['#'])

In [130]:
df_murder = df_murder.rename(columns={"AMOUNT": "murder-100k"})
df_edu = df_edu.rename(columns={"AMOUNT": "highschool-rate"})

df_murder_edu = pd.merge(df_murder, df_edu, on="COUNTRY")

In [131]:
df_murder_edu

,COUNTRY,murder-100k,highschool-rate
0,Brazil,40974.0,81.93
1,India,40752.0,85.13
2,Mexico,25757.0,93.82
3,Ethiopia,20239.0,88.48
4,Indonesia,18963.0,89.65
...,...,...,...
156,Samoa,2.0,96.29
157,Brunei,2.0,99.44
158,Liechtenstein,1.0,98.55
159,Iceland,1.0,99.83


In [132]:
df_murder_edu.corr()

,murder-100k,highschool-rate
murder-100k,1.000000,-0.059328
highschool-rate,-0.059328,1.000000


### REST API
#### Using data from [https://www.energidataservice.dk](https://www.energidataservice.dk) 

In [1]:
import pandas as pd
import requests
from pandas import json_normalize

In [2]:
#get data from an open energy data service provider
url = 'https://www.energidataservice.dk/proxy/api/datastore_search?resource_id=nordpoolmarket&limit=500'

response = requests.get(url)
dictr = response.json() #parse json to dict
recs = dictr['result']['records'] 
df = json_normalize(recs) #flatten json files into data frame
df.head()

,_id,HourUTC,HourDK,PriceArea,SpotPurchase,SpotSale,SpotPriceDKK,ElbasAveragePriceDKK,ElbasMaxPriceDKK,ElbasMinPriceDKK,SpotPriceEUR,ElbasAveragePriceEUR,ElbasMaxPriceEUR,ElbasMinPriceEUR
0,123773,2016-12-31T23:00:00+00:00,2017-01-01T00:00:00,DK2,1080.3,1191.3,155.82,163.428750,173.87625,149.25000,20.96,21.90,23.3,20.0
1,123774,2017-01-01T00:00:00+00:00,2017-01-01T01:00:00,DK2,1027.5,1188.8,155.37,159.473625,180.59250,143.28000,20.90,21.37,24.2,19.2
2,123775,2017-01-01T01:00:00+00:00,2017-01-01T02:00:00,DK2,963.6,1174.6,134.78,111.191250,179.10000,74.62500,18.13,14.90,24.0,10.0
3,123776,2017-01-01T02:00:00+00:00,2017-01-01T03:00:00,DK2,917.6,1145.3,119.17,157.533375,186.56250,134.32500,16.03,21.11,25.0,18.0
4,123777,2017-01-01T03:00:00+00:00,2017-01-01T04:00:00,DK2,883.6,1183.0,122.14,216.337875,216.41250,215.66625,16.43,28.99,29.0,28.9


#### 4
Compute overview statistics (mean, variance, quantiles, counts,...) for all variables. Hint: there is a single pandas call to get this ...

In [3]:
df.describe()

,_id,SpotPurchase,SpotSale,SpotPriceDKK,ElbasAveragePriceDKK,ElbasMaxPriceDKK,ElbasMinPriceDKK,SpotPriceEUR,ElbasAveragePriceEUR,ElbasMaxPriceEUR,ElbasMinPriceEUR
count,500.000000,500.000000,500.000000,500.000000,494.000000,494.000000,494.000000,500.000000,494.000000,494.000000,494.000000
mean,127112.408000,1463.727800,1216.670000,245.378600,217.005792,242.698428,193.717689,33.004900,29.085364,32.528765,25.964170
std,16019.537604,406.768915,375.665708,96.367458,104.673459,125.988858,94.913626,12.958312,14.027668,16.884316,12.719734
min,123773.000000,406.000000,537.100000,-250.880000,-239.015469,0.000000,-357.664510,-33.600000,-32.010000,0.000000,-47.900000
25%,123898.750000,1130.500000,1043.575000,216.372500,191.467256,207.713680,166.224375,29.107500,25.662500,27.850000,22.275000
50%,124023.500000,1503.100000,1170.150000,228.025000,223.759875,232.851840,212.624250,30.670000,29.995000,31.200000,28.500000
75%,124149.250000,1760.900000,1322.250000,260.835000,256.138740,276.658275,238.734400,35.087500,34.320000,37.075000,32.000000
max,281558.000000,3685.700000,3296.100000,870.750000,558.762297,746.310000,502.837700,117.120000,74.870000,100.000000,67.400000


#### 5 
Compute the average ***SpotSale*** by each day. 

In [29]:
df['Date'] = pd.to_datetime(df['HourUTC'])
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

In [32]:
df.groupby('Date')['SpotSale'].mean()

Date
2011-01-01    3296.100000
2011-01-16    2533.750000
2012-01-11    1367.400000
2013-04-10    2719.700000
2016-12-31    1766.000000
2017-01-01    1465.312195
2017-01-02     775.212500
2017-01-03    1054.337500
2017-01-04    1176.920833
2017-01-05    1194.650000
2017-01-06    1019.545833
2017-01-07    1218.637500
2017-01-08     952.587500
2017-01-09    1172.383333
2017-01-10    1273.987500
2017-01-11    1401.754167
2017-01-12    1431.983333
2017-01-13    1161.883333
2017-01-14    1195.700000
2017-01-15    1103.834783
2017-01-16    1287.675000
2017-01-17    1216.921739
2017-01-18    1223.021739
2017-01-19    1073.700000
2017-02-09    2090.800000
2017-12-31     554.600000
2018-01-01    1227.640000
2018-01-02     749.475000
2019-01-02    2800.700000
2020-01-01    3153.000000
Name: SpotSale, dtype: float64

#### 6 
Compute the day with the highest variance in ***SpotPurchase***

In [36]:
df.groupby('Date')['SpotPurchase'].var().idxmax()

'2016-12-31'